In [201]:
# Goal state
goalSt = [ [1, 2, 3], [4, 5, 6], [7, 8, 0] ]

argmin = lambda ls: sorted(ls, key = lambda x: x[0])[0][1]
# argmin recebe uma tupla (x, s) de um valor e um estado.
# faz uma ordenação pelo valor, retornando o estado [1] da
# tupla com menor valor [0]

# Maps a Move to a coordinate
move2coord = {"Up":(-1, 0),
              "Down": (1, 0),
              "Left": (0, -1),
              "Right": (0, 1)}
pos = lambda mv : move2coord[mv]
# O dicionário retorna um vetor/direção do movimento, que será
# somado à posição do qual o agente partiu.

def dist(v, X, Y):
    """
    Calcula a distância entre v no estado dado por X e v no estado
    dado por Y (inicial e final).
    input:
        v(int) : Um número.
        X(array[][]): Um tabuleiro (o atual, p.ex.)
        Y(array[][]): Um tabuleiro (o estado final, p.ex)
    """
    x, y = absDiff( findNum(v, X), findNum(v, Y) )
    return x + y

def absDiff(x, y):
    return ( abs(x[0] - y[0]), abs(x[1] - y[1]) )

def inBound(x):
    if 0 <= x <= 2:
        return True
    return False

# Check if it is a goal state
def isGoal(st):
    s, t = st
    return s == goalSt

# For this puzzle, every state is feasible (There are no constraints)
def feasible(x):
    return True

def addTuple(x, y):
    return (x[0] + y[0], x[1] + y[1])

findZero = lambda x: findNum(0, x)
def findNum(v, X):
    """
    Acha as coordenadas do valor "v" no tabuleiro.
    input:
        v(int) : Valor buscado.
        X(array[][]) : O tabuleiro (matriz) em um dado estado.
    """
    for i, Xi in enumerate(X):
        for j, xij in enumerate(Xi):
            if(xij == v): return (i, j)
            
            
def swap(val, s):
    new_s = [si.copy() for si in s] # faz uma cópia dos elementos de s
    x1, y1 = findNum(val, s) # acha o valor val
    x2, y2 = findZero(s)     # acha o jogador
    # faz a troca
    new_s[x1][y1] = 0        
    new_s[x2][y2] = val
    return new_s

In [202]:
# Update the state
def move(direc, st):
    """
    input:
        direc: Direção do movimento. É obtido pelo dicionário move2coord
        st: Uma tupla (x,y) em que x é o estado atual e y é a sequência
        de movimentos até então feitos para se chegar em x.
    """
    s, t = st
    newX, newY = addTuple(findZero(s), pos(direc))
    # encontra a posicao do jogador (o zero) no estado atual "s"
    # retorna o vetor direcao da escolha "direc"
    # retorna a posicao final do jogador
    if inBound(newX) and inBound(newY): # verifica se a posição é válida
        val = s[newX][newY] # recupera o valor que atualmente
                            # esta na proxima posicao
    else:
        val = 0 # colocando 0, o jogador permanece na mesma posicao
        
    new_s = swap(val, s) # troca o valor da posicao do jogador
                         # (o zero) com o valor da proxima posicao
    return (new_s, t + [direc]) # retorna o novo estado com 
    # um acrescimo na lista de movimentos
    
# perform a move
def moves( sts ):
    """
    Para cada elemento na lista de estados sts,
    input:
        sts(list) : O estado do qual o movimento está partindo
    """
    new_sts = []
    for st in sts:
        choices = move2coord.keys() # retorna as escolhas possiveis
        for c in choices:
            new_sts.append(move(c, st)) # para cada escolha,
            # adiciono o resultado do movimento executado
    return new_sts
    
# Breadth-first search
def bfs( sts ):
    """
    sts é inicial uma tupla com o estado atual e uma lista vazia
    de movimentos.
    """
    if len(sts) == 0:
        print("Couldn't find a feasible solution")
        return ([], []), 0
    
    goal = list(filter(isGoal, sts))
    n = 0 # conta quantas iterações foram necessárias para chegar
          # até a solução
    while len(goal) == 0:
        # dado um estado, executa-se todos os movimentos possiveis
        # e escolhe-se os movimentos válidos (no caso, todos).
        # A árvore de decisões vai se abrindo gradualmente.
        sts = list(filter(feasible, moves(sts)))
        goal = list(filter(isGoal, sts))
        n = n + len(sts) # essa contagem está correta?
    
    return goal[0], n

In [199]:
# Define as heurísticas
def f1(st):
    """
    Soma o custo unitário de cada decisão feita com as distâncias 
    horizontais e verticais de cada peça até sua posição alvo.
    """
    s, t = st
    return len(t) + sum([dist(v, s, goalSt) for v in range(1, 9)])

def f2(st):
    """
    Soma o custo unitário de cada decisão feita com o número de
    peças fora do lugar.
    """
    s, t = st
    return len(t) + sum([s[i][j] != goalSt[i][j]
                        for i in range(3)
                        for j in range(3) ]) - 1

def f3(st):
    """
    Retorna o valor máximo das heurísticas anteriores.
    """
    return max(f1(st), f2(st))


def astar( sts, f ):
    """
    input:
        sts: o estado do qual a busca parte?
        f: A heurística utilizada.
    """
    if len(sts) == 0: # verifica a validade do estado
        print("Couldn't find a feasible solution") 
        return ([], []), 0
    
    goal = list( filter(isGoal, sts) )
    n = 0
    sold = sts[0] # estado anterior
    while len(goal) == 0:
        s = argmin(zip(map(f, sts), sts)) # seleciona o estado com
        # menor custo segundo a heurística
        sts = [si for si in sts if si[0] != s[0]] # não pega estados repetidos (?)
        sts += moves([s]) # adiciona todos os movimentos possiveis partindo de s
        sold = s # isso parece ser inutil
        goal = list(filter(isGoal, sts))
        n = n + len(moves([s])) # essa contagem está correta?
        
    return goal[0], n

In [204]:
s0 = [[1, 8, 2], [0, 4, 3], [7, 6, 5]]

sB, nB = bfs([(s0, [])])
print(f'Busca em largura atingiu o estado {sB[0]} com a sequência {sB[1]} em {nB} jogadas\n\n')

sA1, nA1 = astar([(s0, [])], f1)
print(f'A*-f1 atingiu o estado {sA1[0]} com a sequência {sA1[1]} em {nA1} jogadas\n\n')


Busca em largura atingiu o estado [[1, 2, 3], [4, 5, 6], [7, 8, 0]] com a sequência ['Right', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Down'] em 349524 jogadas


A*-f1 atingiu o estado [[1, 2, 3], [4, 5, 6], [7, 8, 0]] com a sequência ['Right', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Down'] em 44 jogadas




In [228]:
from utils import (is_in, memoize, PriorityQueue)
from collections import deque
from copy import copy

class Problem(object):
    def __init__(self, initial, goal = None):
        self.initial = initial
        self.goal = goal
        
    def actions(self, state):
        raise NotImplementedError
        
    def result(self, state, action):
        raise NotImplementedError
        
    def goal_test(self, state):
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal
        
    def path_cost(self, c, state1, action, state2):
        return c + 1
    
    def value(self, state):
        raise NotImplementedError
        
        
class Node:
    def __init__(self, state, parent=None, action=None, path_cost = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1
    def __repr__(self):
        return "<Node {}>".format(self.state)
    
    def __lt__(self, node):
        return self.state < node.state
    
    def expand(self, problem):
        novosNodos = [self.child_node(problem, action)
                        for action in problem.actions(self.state)]
        return list(filter(lambda nodo: nodo.state != None, novosNodos))
    
    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                         problem.path_cost(self.path_cost, self.state,
                                           action, next_state))
        return next_node
        
    def solution(self):
        return [node.action for node in self.path()[1:]]
    
    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state
    
    def __hash__(self):
        return hash(self.state)

In [274]:
class Queens8(Problem):
    def __init__(self, initialState):
        Problem.__init__(self, initialState)
        
    def actions(self, state):
        posicoes = list()
        for k in range(1, 9):
            posicoes.append([k])
        
        return posicoes
    
    def result(self, state, action):
        new_state = action + state
        if self.movValido(new_state):
            return new_state
        return None
    
    def goal_test(self, state):
        return self.movValido(state) and len(state) == 8
        
    def upperDiag(self, Rainhas):
        return [x+y for x,y in zip(Rainhas, range(1, 9))]
    def lowerDiag(self, Rainhas):
        return [x-y for x,y in zip(Rainhas, range(1, 9))]
    def ataque(self, r, Rainhas):
        return (r in Rainhas) or (r in self.upperDiag(Rainhas)) or \
                                 (r in self.lowerDiag(Rainhas))
    def movValido(self, state):
        return (len(state) == 0) or \
               ( len(state) <= 8 and \
                (not self.ataque(state[0], state[1:])) and \
                self.movValido((state[1:])) )
    
class Puzzle8(Problem):
    def __init__(self, initialState, goalState):
        Problem.__init__(self, initialState, goalState)
        self.move2coord = {"Up":    (-1, 0),
                           "Down":  (1, 0),
                           "Left":  (0, -1),
                           "Right": (0, 1)}
        
    def actions(self, state):
        return list(self.move2coord.keys())
        
    def swap(self, val, state):
        new_state = [list(si).copy() for si in state]
        x1, y1 = self.findNum(val, state)
        x2, y2 = self.findZero(state)
        new_state[x1][y1] = 0
        new_state[x2][y2] = val
        new_state = tuple([tuple(si) for si in new_state])
        return new_state
    
    def addTuple(self, x, y):
        return (x[0] + y[0], x[1] + y[1])
    
    def move(self, direction, state):
        pos = lambda mv: self.move2coord[mv] 
        inBound = lambda x: 0 <= x <= 2
        
        newX, newY = self.addTuple(self.findZero(state), pos(direction))
        if inBound(newX) and inBound(newY):
            val = state[newX][newY]
        else:
            val = 0
        new_state = self.swap(val, state)
        
        return new_state
    
    def result(self, state, action):
        new_state = self.move(action, state)
        if self.movValido(new_state):
            return new_state
        else:
            return None
    
    def movValido(self, state):
        return True
    
    def goal_test(self, state):
        return state == self.goal
    
    def findZero(self, state):
        return self.findNum(0, state)
        
    def findNum(self, n, X):
        for i, xi in enumerate(X):
            for j, xij in enumerate(xi):
                if(xij == n): return (i, j)
        
    def path_cost(self, c, state1, action, state2):
        return c + 1


In [285]:
def depth_first_tree_search(problem):
    frontier = [Node(problem.initial)] # Stack
    n = 0
    while frontier:
        node = frontier.pop()
        n = n + 1
        if problem.goal_test(node.state):
            return (node, n)
        frontier.extend(node.expand(problem))
    return None

def breadth_first_tree_search(problem):
    frontier = deque([Node(problem.initial)])
    n = 0
    while frontier:
        node = frontier.popleft()
        n = n + 1
        if problem.goal_test(node.state):
            return (node, n)
        frontier.extend(node.expand(problem))
    return None

def best_first_graph_search(problem, f):
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = list()
    n = 0
    while frontier:
        node = frontier.pop()
        n = n + 1
        if problem.goal_test(node.state):
            return (node, n)
        explored.append(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

def uniform_cost_search(problem):
    return best_first_graph_search(problem, lambda node : node.path_cost)
    
def astar_search(problem, h = None):
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n))

In [286]:
problem = Queens8([])
sol, n = depth_first_tree_search(problem)
print(sol, n)
sol, n = breadth_first_tree_search(problem)
print(sol, n)
sol, n = uniform_cost_search(problem)
print(sol, n)

<Node [5, 7, 2, 6, 3, 1, 4, 8]> 114
<Node [4, 2, 7, 3, 6, 8, 5, 1]> 1966
<Node [1, 5, 8, 6, 3, 7, 2, 4]> 1966


In [287]:
initial = [[1, 8, 2], [0, 4, 3], [7, 6, 5]]
goal = [ [1, 2, 3], [4, 5, 6], [7, 8, 0] ]
problem = Puzzle8(initial, goal)
sol, n = astar_search(problem, h1)
# print(sol, n)

KeyboardInterrupt: 

In [261]:
def h1(nodo):
    def findNum(n, X):
        for i, xi in enumerate(X):
            for j, xij in enumerate(xi):
                if(xij == n): return (i, j)
    def dist(n, state, goal):
        """
        Calcula a distância entre o número n no estado atual
        e no estado objetivo
        """
        absDiff = lambda x,y: ( abs(x[0] - y[0]), abs(x[1] - y[1]))
        x, y = absDiff(findNum(n, state), findNum(n, goal))
        return x + y
    
    state = nodo.state
    return sum([dist(n, state, goal) for n in range(1, 9)])